In [9]:
import os
import requests
import re
import time
import random

# Liste des ressources minières
resources = [
    "Base metals", "Bauxite", "Chromite", "Coal", "Chromium", "Cobalt", "Coltan", "Copper", "Cuivre", "Diamonds",
    "Ferrotungsten", "Gold", "Graphite", "Heavy Mineral Sands", "Ilmenite", "Iron", "Iron Ore", "Lead", "Lithium",
    "Manganese", "Manganese Ore", "Molybdenum", "Monazite", "Nickel", "Niobium", "Not Specified", "Or", "Phosphate",
    "Platinum Group Metals", "Rare Earth Elements", "Rutile", "Silver", "Tantalum", "Tin", "Titanium",
    "Transition Minerals", "Tungsten", "Uranium (U308)", "Wollastonite", "Zinc", "Zircon"
]

# Liste des pays cibles
countries = ["CD", "AR"]

# Dossier pour enregistrer les fichiers PDF
output_folder = "contracts_pdf"
os.makedirs(output_folder, exist_ok=True)

# URL des API
API_URL = "https://api.resourcecontracts.org/contracts/group"
CONTRACT_METADATA_URL = "https://api.resourcecontracts.org/contract/{}/metadata"

# Longueur maximale des noms de fichiers
MAX_FILENAME_LENGTH = 100

# Fonction pour nettoyer et tronquer les noms de fichiers
def clean_filename(filename):
    cleaned_filename = re.sub(r'[\/:*?"<>|]', "_", filename)  # Supprimer les caractères invalides
    return cleaned_filename[:MAX_FILENAME_LENGTH] if len(cleaned_filename) > MAX_FILENAME_LENGTH else cleaned_filename

# Suivi des résultats de téléchargement
total_found = 0
total_downloaded = 0
missing_contracts = []

# Fonction pour effectuer une requête API de manière sécurisée avec des tentatives de réessai
def safe_request(url, params=None, retries=3):
    for attempt in range(retries):
        try:
            response = requests.get(url, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            wait_time = 2 ** attempt + random.uniform(0, 1)
            print(f"Erreur : {e}. Nouvelle tentative dans {wait_time:.2f} secondes...")
            time.sleep(wait_time)
    return None  # Retourne None si toutes les tentatives échouent

# Fonction pour télécharger les contrats
def download_contracts(country, resource):
    global total_found, total_downloaded

    params = {
        "country_code": country,
        "resource": resource,
        "group": "metadata|text|annotations",
        "per_page": 100,
        "page": 1
    }

    while True:
        data = safe_request(API_URL, params=params)
        if not data:
            print(f"Erreur API pour {country} - {resource}, passage à la suite...")
            break

        contracts = data.get("results", [])
        next_page = data.get("next", None)

        if not contracts:
            print(f"Aucun contrat trouvé pour {country} - {resource}.")
            break

        total_found += len(contracts)
        print(f"✅ {len(contracts)} contrats trouvés pour {country} - {resource} (Page {params['page']}).")

        for contract in contracts:
            contract_id = contract.get("id")
            contract_name = contract.get("name", "N/A")

            if not contract_id:
                print(f"ID manquant pour le contrat : {contract_name}")
                continue

            # Récupérer les métadonnées pour trouver le lien vers le PDF
            metadata_url = CONTRACT_METADATA_URL.format(contract_id)
            metadata = safe_request(metadata_url)

            if not metadata:
                print(f"Aucune métadonnée trouvée pour le contrat : {contract_name} (ID : {contract_id})")
                missing_contracts.append(contract_name)
                continue

            # Extraire l'URL du PDF
            document_url = None
            files = metadata.get("file", [])
            if isinstance(files, list):
                for file_item in files:
                    if file_item.get("media_type") == "application/pdf":
                        document_url = file_item.get("url", "")
                        break

            if not document_url:
                print(f"Aucun lien PDF trouvé pour le contrat : {contract_name} (ID : {contract_id})")
                missing_contracts.append(contract_name)
                continue

            # Générer un nom de fichier sûr
            pdf_filename = f"{contract_id}_{clean_filename(contract_name)}.pdf"
            pdf_path = os.path.join(output_folder, pdf_filename)

            # S'assurer que le dossier existe
            os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

            try:
                # Télécharger le PDF
                print(f"Téléchargement : {contract_name} depuis {document_url}")
                pdf_response = requests.get(document_url, stream=True, timeout=30)
                pdf_response.raise_for_status()

                with open(pdf_path, "wb") as f:
                    for chunk in pdf_response.iter_content(chunk_size=8192):
                        f.write(chunk)

                total_downloaded += 1
                print(f"Sauvegardé : {pdf_path}")

                # Délai entre les téléchargements pour éviter de dépasser les limites
                time.sleep(2)

            except requests.exceptions.RequestException as e:
                print(f"Échec du téléchargement pour {document_url} : {e}")
                missing_contracts.append(contract_name)

        # Gérer correctement la pagination
        if next_page:
            params["page"] += 1
        else:
            break

# Télécharger les contrats pour chaque pays et ressource
for country in countries:
    for resource in resources:
        print(f"Recherche de contrats pour {country} - {resource}...")
        download_contracts(country, resource)

# Vérifier le compte final
downloaded_files = os.listdir(output_folder)

print("\nRÉSUMÉ FINAL")
print(f"Nombre total de contrats trouvés : {total_found}")
print(f"Nombre de téléchargements réussis : {total_downloaded}")
print(f"Nombre de téléchargements manquants : {len(missing_contracts)}")

# Afficher les contrats manquants
if missing_contracts:
    print("\nContrats non téléchargés :")
    for missing in missing_contracts:
        print(f"  - {missing}")

print("Processus de téléchargement terminé !")


Recherche de contrats pour CD - Base metals...
✅ 1 contrats trouvés pour CD - Base metals (Page 1).
Téléchargement : EMK-Mn, Cluff Mining Limited, Concession, 1998 depuis https://resourcecontracts-nrgi.s3-us-west-2.amazonaws.com/865/865-emk-mn-drc-1998.pdf
Sauvegardé : contracts_pdf/865_EMK-Mn, Cluff Mining Limited, Concession, 1998.pdf
Recherche de contrats pour CD - Bauxite...
Aucun contrat trouvé pour CD - Bauxite.
Recherche de contrats pour CD - Chromite...
Aucun contrat trouvé pour CD - Chromite.
Recherche de contrats pour CD - Coal...
✅ 1 contrats trouvés pour CD - Coal (Page 1).
Téléchargement : Gécamines, La Grande Cimenterie du Katanga, PE466, Partial Amodiation, 2020 depuis https://resourcecontracts-nrgi-prod.s3.amazonaws.com/5223/5223-gecamines-contrat-damodiation-partielle-avec-la-grande-cimenterie-du-katanga.pdf
Sauvegardé : contracts_pdf/5223_Gécamines, La Grande Cimenterie du Katanga, PE466, Partial Amodiation, 2020.pdf
Recherche de contrats pour CD - Chromium...
✅ 1 con

In [8]:
import shutil
from google.colab import files

# Nom du dossier contenant les PDFs
folder_name = "contracts_pdf"

# Nom du fichier ZIP
zip_filename = f"{folder_name}.zip"

# Compresser le dossier en ZIP
shutil.make_archive(folder_name, 'zip', folder_name)

# Télécharger le fichier ZIP
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>